In [3]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import logging
from typing import Dict

In [4]:
# Diccionario para los tipos de la data
dict_detalle = {"Referencia":"str",
                "NIT":"str",
                "Lib.mayor":"str",
                "Div.":"str",
                "CeBe":"str",
                "VP":"str",
                "Asignación":"str",
                "Texto":"str",
                "Clave ref.1":"str",
                "Clave ref.2":"str",
                "Clave referencia 3":"str",
                "Fec.Expedi":"str",
                "F.FinVigen":"str",
                "Importe":"str",
                "Fe.valor":"str"}

dict_cabecera = {"Referencia":"str",
                 "Texto cab.documento":"str",
                 "Anul.con":"str",
                 "Tipo cambio":"str"}

In [19]:
archivo_input = "01.Input\Detalle\Detalle.xlsx"
archivo_cabecera = "01.Input\Cabecera\Cabecera.xlsx"
df_importado = pd.read_excel(archivo_input, dtype=dict_detalle)
df_cabecera = pd.read_excel(archivo_cabecera, dtype=dict_cabecera)

In [20]:
# Cruce de df_importado con df_cabecera
df_importado = df_importado.merge(df_cabecera, on='Referencia', how='inner')

# Descartar referencias que ya hayan sido corregidas
df_importado = df_importado.loc[df_importado['Texto cab.documento'] != 'ACTUALIZA TRM']

In [21]:
# Transformaciones previas a la validación
df_importado['Referencia_temp'] = df_importado['Referencia'].apply(lambda x: x[:-1] if x.endswith('I') else x)
df_importado['Tipo cambio'] = df_importado['Tipo cambio'].apply(lambda x: round(float(f"{x.replace('.', '').replace(',', '')[:-5]}.{x.replace('.', '').replace(',', '')[-5:]}"), 2))

In [22]:
# Validar referencias que tienen producción e IVA calculado con diferente TRM
trm_unicas = df_importado.groupby('Referencia_temp').agg(TRM_Unicas=('Tipo cambio', 'nunique')).reset_index()

# Filtrar solo registros de IVA para asignar la tasa correcta
trm_iva = df_importado.loc[df_importado['Referencia'].str.endswith('I')].drop_duplicates(subset=['Referencia'])[['Referencia_temp','Tipo cambio']]
trm_iva = trm_iva.rename(columns={'Tipo cambio': 'Tasa_Correcta'})

In [23]:
# Cruce de df_importado con trm_unicas para llevar la columna TRM_Unicas a la BD principal
df_importado = df_importado.merge(trm_unicas, on='Referencia_temp', how='left')

# Cruce de df_importado con trm_iva para asignar la tasa correcta
df_importado = df_importado.merge(trm_iva, on='Referencia_temp', how='left')

# Corregir columna Tasa_Correcta cuando la referencia no tenga IVA
df_importado['Tasa_Correcta'] = df_importado.apply(lambda x: x['Tipo cambio'] if pd.isna(x['Tasa_Correcta']) else x['Tasa_Correcta'], axis=1)

In [24]:
# Filtrar aquellas referencias que tienen TRM diferentes
df_depurado = df_importado.loc[df_importado['TRM_Unicas'] != 1]
df_depurado = df_depurado.sort_values(by=['Referencia', 'Lib.mayor'])

In [49]:
# Definición de parámatros estandar
agrupador = 1
fecha_documento = datetime.now()
clase_documento = 'SL'
sociedad =  1000
periodo = fecha_documento.strftime("%m")
clave_moneda = 'USD'
txt_cabecera = 'ACTUALIZAR TRM'
indicador_IVA = 'GA'
# Generación de plantilla para Notas Crédito
df_plantilla1 = df_depurado
df_plantilla1 = df_plantilla1.rename(columns={'Tipo cambio': 'Tipo de Cambio'})
df_plantilla1 = df_plantilla1.rename(columns={'Referencia': 'Número documento de referencia'})
df_plantilla1 = df_plantilla1.rename(columns={'Lib.mayor': 'Cuenta de mayor de la contabilidad principal'})
df_plantilla1 = df_plantilla1.rename(columns={'NIT': 'Nit'})
df_plantilla1['Importe'] = df_plantilla1['Importe'].astype(float).abs()
df_plantilla1 = df_plantilla1.rename(columns={'Importe': 'Importe en la moneda del documento'})
df_plantilla1 = df_plantilla1.rename(columns={'Div.': 'División'})
df_plantilla1 = df_plantilla1.rename(columns={'CeBe': 'Centro de coste'})
df_plantilla1 = df_plantilla1.rename(columns={'VP': 'Vía de pago'})
df_plantilla1 = df_plantilla1.rename(columns={'Asignación': 'Número de asignación'})
df_plantilla1 = df_plantilla1.rename(columns={'Texto': 'Texto posición'})
df_plantilla1 = df_plantilla1.rename(columns={'Clave ref.1': 'Clave de referencia de interlocutor comercial'})
df_plantilla1 = df_plantilla1.rename(columns={'Clave ref.2': 'Clave de referencia de interlocutor comercial 2'})
df_plantilla1 = df_plantilla1.rename(columns={'Clave referencia 3': 'Clave de referencia para la posición de documento'})
df_plantilla1 = df_plantilla1.rename(columns={'Fec.Expedi': 'Fecha expeENEion'})
df_plantilla1 = df_plantilla1.rename(columns={'F.FinVigen': 'Fecha fin vigencia'})
df_plantilla1['Agrupdor de Documentos'] = agrupador
df_plantilla1['Fecha de Documento'] = fecha_documento
df_plantilla1['Clase de Documento'] = clase_documento
df_plantilla1['Sociedad'] = sociedad
df_plantilla1['Fecha Contabilización'] = fecha_documento
df_plantilla1['Período'] = periodo
df_plantilla1['Clave de Moneda'] = clave_moneda
df_plantilla1['Fe.conversión '] = ""
df_plantilla1['Texto de Cebecera'] = txt_cabecera
df_plantilla1['Calc. Impuestos '] = np.select([(df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "4101101000")],                    
                                              ["X"],
                                              default="")
df_plantilla1['Clave de contabilización para la siguiente posición'] = np.select([(df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "1074010000") | (df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "1074103000"),
                                                                                  (df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "4101101000") | (df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "2271021622")],
                                                                                 ["11",
                                                                                  "40"],
                                                                                 default="Revisar")
df_plantilla1['Número de identificación fiscal 1'] = np.select([(df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "1074010000") | (df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "1074103000")],
                                                               [df_plantilla1['Nit']],
                                                               default="")
df_plantilla1['Tipo de número de identificación fiscal'] = np.select([(df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "1074010000") | (df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "1074103000")],
                                                                     ["31"],
                                                                     default="")
df_plantilla1['In. CME'] = ""
df_plantilla1['Indicador IVA'] = indicador_IVA
df_plantilla1['InENEador retefuente WT_WITHCD'] = ""
df_plantilla1['TP retefuente WITHT'] = ""
df_plantilla1['inENEador reteica WT_WITHCD'] = ""
df_plantilla1['TP reteica WITHT'] = ""
df_plantilla1['inENEador reteiva WT_WITHCD'] = ""
df_plantilla1['TP reteiva WITHT'] = ""
df_plantilla1['inENEador retetimbre WT_WITHCD'] = ""
df_plantilla1['Tp retetimbre WITHT'] = ""
df_plantilla1['Clave de conENEiones de pago'] = ""
df_plantilla1['Centro de coste'] = np.select([(df_plantilla1['Cuenta de mayor de la contabilidad principal'] == "4101101000")],                    
                                             [df_plantilla1['Centro de coste']],
                                             default="")
df_plantilla1['Centro de beneficio'] = ""
df_plantilla1['Días del descuento por pronto pago 1'] = ""
df_plantilla1['Porcentaje de descuento por pronto pago 1'] = ""
df_plantilla1['Días del descuento por pronto pago 2'] = ""
df_plantilla1['Porcentaje de descuento por pronto pago 2'] = ""
df_plantilla1['Plazo para conENEión de pago neto'] = ""
df_plantilla1['Fecha valor'] = "PENDIENTE"
df_plantilla1['Fecha base para cálculo del vencimiento'] = "PENDIENTE"
df_plantilla1['ConENEión de pago fija'] = ""
df_plantilla1['Base de descuento'] = ""
df_plantilla1['Bloqueo pago'] = ""
df_plantilla1['% DPP'] = ""
df_plantilla1['Importe DPP'] = ""
df_plantilla1['Número de orden'] = ""
df_plantilla1['Tipo de banco interlocutor'] = ""
df_plantilla1['Poliza líder'] = ""
df_plantilla1['Cert.lider'] = ""
df_plantilla1['Nombre'] = ""

# Generación de plantilla para Corrección
df_plantilla2 = df_depurado
df_plantilla2 = df_plantilla2.rename(columns={'Tasa_Correcta': 'Tipo de Cambio'})
df_plantilla2 = df_plantilla2.rename(columns={'Referencia': 'Número documento de referencia'})
df_plantilla2 = df_plantilla2.rename(columns={'Lib.mayor': 'Cuenta de mayor de la contabilidad principal'})
df_plantilla2 = df_plantilla2.rename(columns={'NIT': 'Nit'})
df_plantilla2['Importe'] = df_plantilla2['Importe'].astype(float).abs()
df_plantilla2 = df_plantilla2.rename(columns={'Importe': 'Importe en la moneda del documento'})
df_plantilla2 = df_plantilla2.rename(columns={'Div.': 'División'})
df_plantilla2 = df_plantilla2.rename(columns={'CeBe': 'Centro de coste'})
df_plantilla2 = df_plantilla2.rename(columns={'VP': 'Vía de pago'})
df_plantilla2 = df_plantilla2.rename(columns={'Asignación': 'Número de asignación'})
df_plantilla2 = df_plantilla2.rename(columns={'Texto': 'Texto posición'})
df_plantilla2 = df_plantilla2.rename(columns={'Clave ref.1': 'Clave de referencia de interlocutor comercial'})
df_plantilla2 = df_plantilla2.rename(columns={'Clave ref.2': 'Clave de referencia de interlocutor comercial 2'})
df_plantilla2 = df_plantilla2.rename(columns={'Clave referencia 3': 'Clave de referencia para la posición de documento'})
df_plantilla2 = df_plantilla2.rename(columns={'Fec.Expedi': 'Fecha expeENEion'})
df_plantilla2 = df_plantilla2.rename(columns={'F.FinVigen': 'Fecha fin vigencia'})
df_plantilla2['Agrupdor de Documentos'] = agrupador
df_plantilla2['Fecha de Documento'] = fecha_documento
df_plantilla2['Clase de Documento'] = clase_documento
df_plantilla2['Sociedad'] = sociedad
df_plantilla2['Fecha Contabilización'] = fecha_documento
df_plantilla2['Período'] = periodo
df_plantilla2['Clave de Moneda'] = clave_moneda
df_plantilla2['Fe.conversión '] = ""
df_plantilla2['Texto de Cebecera'] = txt_cabecera
df_plantilla2['Calc. Impuestos '] = np.select([(df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "4101101000")],                    
                                              ["X"],
                                              default="")
df_plantilla2['Clave de contabilización para la siguiente posición'] = np.select([(df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "1074010000") | (df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "1074103000"),
                                                                                  (df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "4101101000") | (df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "2271021622")],
                                                                                 ["01",
                                                                                  "50"],
                                                                                 default="Revisar")
df_plantilla2['Número de identificación fiscal 1'] = np.select([(df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "1074010000") | (df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "1074103000")],
                                                               [df_plantilla2['Nit']],
                                                               default="")
df_plantilla2['Tipo de número de identificación fiscal'] = np.select([(df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "1074010000") | (df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "1074103000")],
                                                                     ["31"],
                                                                     default="")
df_plantilla2['In. CME'] = ""
df_plantilla2['Indicador IVA'] = indicador_IVA
df_plantilla2['InENEador retefuente WT_WITHCD'] = ""
df_plantilla2['TP retefuente WITHT'] = ""
df_plantilla2['inENEador reteica WT_WITHCD'] = ""
df_plantilla2['TP reteica WITHT'] = ""
df_plantilla2['inENEador reteiva WT_WITHCD'] = ""
df_plantilla2['TP reteiva WITHT'] = ""
df_plantilla2['inENEador retetimbre WT_WITHCD'] = ""
df_plantilla2['Tp retetimbre WITHT'] = ""
df_plantilla2['Clave de conENEiones de pago'] = ""
df_plantilla2['Centro de coste'] = np.select([(df_plantilla2['Cuenta de mayor de la contabilidad principal'] == "4101101000")],                    
                                             [df_plantilla2['Centro de coste']],
                                             default="")
df_plantilla2['Centro de beneficio'] = ""
df_plantilla2['Días del descuento por pronto pago 1'] = ""
df_plantilla2['Porcentaje de descuento por pronto pago 1'] = ""
df_plantilla2['Días del descuento por pronto pago 2'] = ""
df_plantilla2['Porcentaje de descuento por pronto pago 2'] = ""
df_plantilla2['Plazo para conENEión de pago neto'] = ""
df_plantilla2['Fecha valor'] = "PENDIENTE"
df_plantilla2['Fecha base para cálculo del vencimiento'] = "PENDIENTE"
df_plantilla2['ConENEión de pago fija'] = ""
df_plantilla2['Base de descuento'] = ""
df_plantilla2['Bloqueo pago'] = ""
df_plantilla2['% DPP'] = ""
df_plantilla2['Importe DPP'] = ""
df_plantilla2['Número de orden'] = ""
df_plantilla2['Tipo de banco interlocutor'] = ""
df_plantilla2['Poliza líder'] = ""
df_plantilla2['Cert.lider'] = ""
df_plantilla2['Nombre'] = ""

estructura_final =["Agrupdor de Documentos",
                   "Fecha de Documento",
                   "Clase de Documento",
                   "Sociedad",
                   "Fecha Contabilización",
                   "Período",
                   "Clave de Moneda",
                   "Tipo de Cambio",
                   "Fe.conversión ",
                   "Número documento de referencia",
                   "Texto de Cebecera",
                   "Calc. Impuestos ",
                   "Clave de contabilización para la siguiente posición",
                   "Número de identificación fiscal 1",
                   "Tipo de número de identificación fiscal",
                   "In. CME",
                   "Cuenta de mayor de la contabilidad principal",
                   "Importe en la moneda del documento",
                   "Indicador IVA",
                   "InENEador retefuente WT_WITHCD",
                   "TP retefuente WITHT",
                   "inENEador reteica WT_WITHCD",
                   "TP reteica WITHT",
                   "inENEador reteiva WT_WITHCD",
                   "TP reteiva WITHT",
                   "inENEador retetimbre WT_WITHCD",
                   "Tp retetimbre WITHT",
                   "División",
                   "Clave de conENEiones de pago",
                   "Centro de coste",
                   "Centro de beneficio",
                   "Días del descuento por pronto pago 1",
                   "Porcentaje de descuento por pronto pago 1",
                   "Días del descuento por pronto pago 2",
                   "Porcentaje de descuento por pronto pago 2",
                   "Plazo para conENEión de pago neto",
                   "Fecha valor",
                   "Fecha base para cálculo del vencimiento",
                   "ConENEión de pago fija",
                   "Base de descuento",
                   "Vía de pago",
                   "Bloqueo pago",
                   "% DPP",
                   "Importe DPP",
                   "Número de asignación",
                   "Texto posición",
                   "Número de orden",
                   "Clave de referencia de interlocutor comercial",
                   "Clave de referencia de interlocutor comercial 2",
                   "Clave de referencia para la posición de documento",
                   "Tipo de banco interlocutor",
                   "Fecha expeENEion",
                   "Fecha fin vigencia",
                   "Poliza líder",
                   "Cert.lider",
                   "Nit",
                   "Nombre"]

df_plantilla1 = df_plantilla1[estructura_final]
df_plantilla2 = df_plantilla2[estructura_final]

In [51]:
df_plantilla1.to_excel('Prueba.xlsx', index=False)

In [50]:
df_plantilla2.to_excel('Prueba2.xlsx', index=False)